# Testing PEFT configurations on a subset of the training data

In [1]:
from datasets import load_dataset
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..'))) 
from src.helper_functions import format_mcf_finetuning


# Load model
usml_raw = load_dataset("GBaker/MedQA-USMLE-4-options")
usml_train = usml_raw['train']
sample_train = usml_train.shuffle(seed=42).select(range(100))
print(sample_train)

Dataset({
    features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
    num_rows: 100
})


Preprocess the subset

In [2]:
formatted_train_subset = sample_train.map(
    format_mcf_finetuning,
    remove_columns=sample_train.column_names
)

print(formatted_train_subset[0]['prompt'])
print(formatted_train_subset[0]['completion'])

Question: A 35-year-old woman comes to your office with a variety of complaints. As part of her evaluation, she undergoes laboratory testing which reveals the presence of anti-centromere antibodies. All of the following symptoms and signs would be expected to be present EXCEPT:
A. Pallor, cyanosis, and erythema of the hands
B. Blanching vascular abnormalities
C. Hypercoagulable state
D. Heartburn and regurgitation
Answer:
Hypercoagulable state


In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    Trainer
)
from peft import LoraConfig, get_peft_model

# 1. GPU Verification
assert torch.cuda.is_available(), "GPU not detected!"
print(f"CUDA version: {torch.version.cuda}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")
torch.cuda.empty_cache()

# 2. Tokenizer Setup
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# 3. Tokenization Function
def tokenize_function(examples):
    texts = [p + c for p, c in zip(examples['prompt'], examples['completion'])]
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Create labels (mask prompt tokens)
    prompt_lens = [len(tokenizer(p)['input_ids']) for p in examples['prompt']]
    labels = tokenized["input_ids"].clone()
    for i, plen in enumerate(prompt_lens):
        labels[i, :plen] = -100
    
    tokenized["labels"] = labels
    return tokenized

# 4. Apply tokenization
tokenized_dataset = formatted_train_subset.map(
    tokenize_function,
    batched=True,
    remove_columns=['prompt', 'completion'],
    batch_size=8  # Smaller batches for tokenization
)

# 5. Model Loading (FP16)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=True
)

# 6. LoRA Configuration (Standard, no quantization prep)
lora_config = LoraConfig(
    r=8,  # Reduced from 16 for memory
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 7. Verify parameters
model.print_trainable_parameters()  # Should show ~0.1% of parameters

# 8. Training Arguments (Memory-optimized)
training_args = TrainingArguments(
    output_dir="./llama7b-lora-standard",
    per_device_train_batch_size=2,  # Reduced from 4
    gradient_accumulation_steps=8,  # Increased to compensate
    warmup_ratio=0.1,
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    optim="adamw_torch",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to="none"
)

# 9. Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

# 10. Pre-Training Memory Check
print(f"Pre-train VRAM: {torch.cuda.memory_allocated()/1e9:.2f}GB used")

# 11. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

CUDA version: 12.6
VRAM: 23.57GB


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622
Pre-train VRAM: 13.49GB used


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,7.308500
20,7.436800


TrainOutput(global_step=21, training_loss=7.216925893511091, metrics={'train_runtime': 155.9321, 'train_samples_per_second': 1.924, 'train_steps_per_second': 0.135, 'total_flos': 6093193347072000.0, 'train_loss': 7.216925893511091, 'epoch': 3.0})

In [ ]:
# 1. Save adapters (PEFT automatically excludes base model)
model.save_pretrained(
    "llama7b-usmle-lora-subset",
    safe_serialization=True,  # Uses modern .safetensors format
    max_shard_size="200MB"  # Optional: splits large adapters
)

Now push the fine tuned model to Hugging Face.

In [6]:
# 1. Define your custom model name
MODEL_NAME = "usmle-llama7b-lora-subset"  
USERNAME = "jihbr"  # Your Hugging Face username

# 2. Verify critical files
import os
required_files = {
    "adapter_config.json": "LoRA configuration",
    "adapter_model.safetensors": "Adapter weights"
}
for file, desc in required_files.items():
    assert os.path.exists(f"llama7b-usmle-lora-subset/{file}"), f"Missing {desc} file"

# 3. Accelerated upload (5x faster in GCP)
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# 4. Upload with custom name
from huggingface_hub import HfApi, create_repo

# Create the repository with your custom name
repo_url = create_repo(
    repo_id=f"{USERNAME}/{MODEL_NAME}",  # Changed here
    repo_type="model",
    token=True,
    exist_ok=True
)
print(f"Created repository: {repo_url}")

# Upload to your named repository
HfApi().upload_folder(
    folder_path="llama7b-usmle-lora-subset",
    repo_id=f"{USERNAME}/{MODEL_NAME}",  # Changed here
    repo_type="model",
    commit_message="Add USMLE LoRA adapters - Medical QA Fine-Tune",
    ignore_patterns=["*.bin", "pytorch_model*"],
    allow_patterns=["adapter_*", "README*"]
)

print("✅ Adapters uploaded successfully to:")
print(f"https://huggingface.co/{USERNAME}/{MODEL_NAME}")  # Changed here

Created repository: https://huggingface.co/jihbr/usmle-llama7b-lora-subset


Uploading...:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

✅ Adapters uploaded successfully to:
https://huggingface.co/jihbr/usmle-llama7b-lora-subset
